# CREACIÓN DE RDF

In [1]:
!pip install morph-kgc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.4/113.4 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: duckdb
    Found existing installation: duckdb 0.9.2
    Uninstalling duckdb-0.

In [2]:
import morph_kgc

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
config1="""
          [/content/drive/MyDrive/hands_on_mapping/parking-paris-updated-with-links.csv]
          mappings: /content/drive/MyDrive/hands_on_mapping/mapeo_parking_with_links.rml.ttl
       """
config2="""
          [/content/drive/MyDrive/hands_on_mapping/adresse-paris-updated-with-links.csv]
          mappings: /content/drive/MyDrive/hands_on_mapping/mapeo_streets_with_links.rml.ttl
       """

In [5]:
g1 = morph_kgc.materialize(config1)
g2 = morph_kgc.materialize(config2)
g = g1 +g2

/usr/local/lib/python3.10/dist-packages/morph_kgc/mapping/mapping_parser.py:390: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  fnml_df = fnml_df.applymap(str)
/usr/local/lib/python3.10/dist-packages/morph_kgc/mapping/mapping_partitioner.py:182: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f"{self.rml_df['mapping_partition'].value_counts()[0]}.")
/usr/local/lib/python3.10/dist-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/usr/local/lib/python3.10/dist-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/usr/local/lib/python3.10/dist-packages/morph_kgc/materializer.py

In [6]:
v1 = g1.serialize(destination='/content/drive/MyDrive/hands_on_mapping/parkings_with_linksRDF.ttl', format="ttl")
v2 = g2.serialize(destination='/content/drive/MyDrive/hands_on_mapping/streets_with_linksRDF.ttl', format="ttl")
v = g.serialize(destination='/content/drive/MyDrive/hands_on_mapping/combined_with_linksRDF.ttl', format="ttl")
v

<Graph identifier=N8a7bc9e6f663403d8dac59e5932c1ee5 (<class 'rdflib.graph.Graph'>)>

# QUERYS



In [7]:
from rdflib import Graph, Namespace, RDF, RDFS
from rdflib.plugins.sparql import prepareQuery

ns = Namespace("http://paris.linkeddata.es/info/resource/parkings#")
ns2 = Namespace('http://paris.linkeddata.es/info/resource/streets#')

geo = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")

### 1. DADO UN DISTRITO DE WIKIDATA, DEVOLVER EL NOMBRE DEL PARKING Y SUS COORDENADAS

In [8]:
q1 = prepareQuery('''
    SELECT ?name ?lat ?lon
    WHERE {
      ?parking a ns:Parking;
              ns:isinDistrict ?d;
              ns:hasName ?name;
              ns:hasGeoPoint ?point.
      ?d owl:sameAs <https://wikidata.org/entity/Q171689>.
      ?point geo:lat ?lat;
              geo:long ?lon.


    }
''',
initNs={"ns": ns, 'geo':geo})

for s in g.query(q1):
    print(f'-{s.name}:   ({s.lat}, {s.lon})\n')

-HECTOR MALOT:   (237.633.192, 4.884.690.131)

-MARIGNY CHATEAU:   (243.953.727, 4.884.535.357)

-BERCY RELAIS MERCURE:   (238.153.136, 4.883.817.784)

-FAUBOURG SAINT ANTOINE:   (237.556.129, 4.885.084.205)

-MEDITERRANEE:   (237.574.679, 4.884.497.657)

-REUILLY DIDEROT:   (238.725.815, 4.884.654.591)

-PICPUS NATION:   (239.914.441, 4.884.715.454)

-BERCY SEINE VEHICULES LEGERS:   (238.083.038, 4.883.505.677)

-BASTILLE:   (236.818.245, 4.884.948.604)

-VELOSTATION VAN GOGH:   (237.125.675, 4.884.369.509)

-VAN GOGH:   (237.185.347, 4.884.467.523)

-LYON DIDEROT:   (237.216.992, 4.884.468.165)



### 2. DADO UN TELÉFONO DE CONTACTO DE UN PARKING. HALLAR EL LINK DE WIKIDATA DEL DISTRITO EN EL QUE SE ENCUENTRA, Y EL NOMBRE DEL PARKING

In [9]:
q2 = prepareQuery('''
    SELECT ?d_wiki ?name
    WHERE {
      ?parking a ns:Parking;
              ns:hasName ?name;
              ns:isinDistrict ?d;
              ns:canBeContactedWith ?c.
      ?c ns:hasPhoneNumber 142365737.
      ?d owl:sameAs ?d_wiki.
    }
''',
initNs={"ns": ns})

for s in g.query(q2):
  print(f'Parking: {s["name"]}\n  District: {s["d_wiki"]}')

Parking: LES HALLES ST EUSTACHE
  District: https://wikidata.org/entity/Q161741


### 3. DADO UN DISTRITO, DEVOLVER TODOS LOS PARKINGS CON SUBSRATE QUE SE ENCUENTREN EN ÉL, Y LAS TARIFAS MENSUALES DE TODOS LOS TIPOS DE VEHÍCULO QUE PRESENTAN

In [11]:
q3 = prepareQuery('''
    SELECT ?parking ?carTariff ?motoTariff ?PRMTariff ?electricTariff ?biciTariff
    WHERE {
      ?parking ns:hasSubsRate ?rate;
              ns:isinDistrict ?d.
      ?d owl:sameAs <https://wikidata.org/entity/Q270230>.
      ?rate ns:hasMonthlySubsCarRate ?carTariff;
            ns:hasMonthlySubsMotoRate ?motoTariff;
            ns:hasMonthlySubsPRMRate ?PRMTariff;
            ns:hasMonthlySubsElectricRate ?electricTariff;
            ns:hasMonthlySubsBiciRate ?biciTariff.
    FILTER (?carTariff > 0 && ?motoTariff > 0 && ?PRMTariff > 0
                  && ?electricTariff > 0 && ?biciTariff > 0)
    }
  ''',
  initNs = {"ns": ns}
)

for s in g.query(q3):
    print(f'Parking: {s["parking"]}')
    print(f'  Monthly Subs Car Tariff: {s["carTariff"]}')
    print(f'  Monthly Subs Moto Tariff: {s["motoTariff"]}')
    print(f'  Monthly Subs PRM Tariff: {s["PRMTariff"]}')
    print(f'  Monthly Subs Electric Tariff: {s["electricTariff"]}')
    print(f'  Monthly Subs Bici Tariff: {s["biciTariff"]}\n')


Parking: http://paris.linkeddata.es/info/resource/Parking/75108-P-125
  Monthly Subs Car Tariff: 304.0
  Monthly Subs Moto Tariff: 106.0
  Monthly Subs PRM Tariff: 152.0
  Monthly Subs Electric Tariff: 152.0
  Monthly Subs Bici Tariff: 10.0

Parking: http://paris.linkeddata.es/info/resource/Parking/75108-P-054
  Monthly Subs Car Tariff: 300.0
  Monthly Subs Moto Tariff: 105.0
  Monthly Subs PRM Tariff: 150.0
  Monthly Subs Electric Tariff: 150.0
  Monthly Subs Bici Tariff: 10.0

Parking: http://paris.linkeddata.es/info/resource/Parking/75108-P-030
  Monthly Subs Car Tariff: 293.0
  Monthly Subs Moto Tariff: 102.0
  Monthly Subs PRM Tariff: 146.0
  Monthly Subs Electric Tariff: 205.1
  Monthly Subs Bici Tariff: 12.0

Parking: http://paris.linkeddata.es/info/resource/Parking/75108-P-061
  Monthly Subs Car Tariff: 281.0
  Monthly Subs Moto Tariff: 98.0
  Monthly Subs PRM Tariff: 140.0
  Monthly Subs Electric Tariff: 140.0
  Monthly Subs Bici Tariff: 10.0

Parking: http://paris.linkeddata.